In [5]:
!pip install opencv-python

import cv2

In [2]:
!pip install tensorflow

In [6]:
import tensorflow as tf

In [7]:
import numpy as np
import pandas as pd

In [8]:
interpreter  = tf.lite.Interpreter(model_path='D:\pythonGUI.py\lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors() #have to do when working on tflite model 

## TUTORIAL:

In [10]:
cap2 = cv2.VideoCapture(0)
while cap2.isOpened():
    r,f = cap2.read()
    
    cv2.imshow("example",f)

    if cv2.waitKey(10) & 0xFF==ord('q'):
        break

cap2.release()
cv2.destroyAllWindows()


In [11]:
i= f.copy()
i.shape

(480, 640, 3)

In [13]:
np.expand_dims(i,axis=0).shape

(1, 480, 640, 3)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(i)

In [18]:
photo = tf.image.resize_with_pad(np.expand_dims(i,axis=0),192,192)
photo.shape

TensorShape([1, 192, 192, 3])

In [ ]:
plt.imshow(tf.cast(np.squeeze(photo), dtype=tf.int32)) #cast the array to int and squeeze to an image of size 192,192,3

In [31]:
interpreter.get_input_details()

[{'name': 'serving_default_input:0',
  'index': 0,
  'shape': array([  1, 192, 192,   3]),
  'shape_signature': array([  1, 192, 192,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [29]:
interpreter.get_output_details()[0] #17 features output

{'name': 'StatefulPartitionedCall:0',
 'index': 312,
 'shape': array([ 1,  1, 17,  3]),
 'shape_signature': array([ 1,  1, 17,  3]),
 'dtype': numpy.float32,
 'quantization': (0.0, 0),
 'quantization_parameters': {'scales': array([], dtype=float32),
  'zero_points': array([], dtype=int32),
  'quantized_dimension': 0},
 'sparsity_parameters': {}}

In [32]:
print(keypoints_with_scores)

#The first two channels of the last dimension represents the yx coordinates (normalized to image frame, i.e. range in [0.0, 1.0]) of the 17 keypoints (in the order of: [nose, left eye, right eye, left ear, right ear, left shoulder, right shoulder, left elbow, right elbow, left wrist, right wrist, left hip, right hip, left knee, right knee, left ankle, right ankle]).

[[[[0.42466435 0.6587705  0.6489544 ]
   [0.35407913 0.70092976 0.6238707 ]
   [0.35589737 0.5987868  0.52777934]
   [0.35738382 0.7333615  0.6152539 ]
   [0.36249042 0.50439173 0.72077703]
   [0.5331534  0.81061715 0.8532808 ]
   [0.54145646 0.3879884  0.75764453]
   [0.864633   0.89563406 0.48185226]
   [0.8843539  0.38622272 0.139885  ]
   [0.88623464 0.9149751  0.03409393]
   [0.856852   0.5084202  0.00916915]
   [0.45534155 0.6661201  0.02728752]
   [0.92069596 0.49639106 0.03598105]
   [0.878385   0.88751495 0.01400092]
   [0.5515812  0.3652404  0.02447071]
   [0.7389878  1.0170022  0.00569352]
   [0.42301103 0.14629337 0.011743  ]]]]


the 3rd column represents the confidence matrix results in the predictions made by the model. so down we go we find the confidence points to decrease and that because it is not able to see those parts in the image/frame captured in the video.

In [33]:
right_eye=keypoints_with_scores[0][0][2]

In [34]:
right_eye

array([0.35589737, 0.5987868 , 0.52777934], dtype=float32)

In [35]:
np.array(right_eye[:2]*[480,640]).astype(int) #actual position of the right eye detected by the model wrt to the input image

array([170, 383])

## Actual Model:

In [16]:
cap = cv2.VideoCapture(0) #open web-cam 
while cap.isOpened():
    ret, frame = cap.read() #reads image/frame(480,640,3) and outputs us the ret and frame
    if frame is None: #error correction [stack-overflow]
        break
    
    # Reshape image
    img = frame.copy() #frame -> last image/frame from the video
    #np.expand_dims()-> encapsulate in another array =shape now is (1,480,640,3)
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192) #resize to (192,192,3) also padded.
    input_image = tf.cast(img, dtype=tf.float32) #we need to input float 32 type data into the model.
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()#shown above example
    
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image)) #passing input_image array to the index of the interpreter get_input_details
    interpreter.invoke() #this will go ahead and make our predictions
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index']) #storing our predictions in a variable
    
    # Rendering 
    draw_connections(frame, keypoints_with_scores, EDGES, 0.1)
    draw_keypoints(frame, keypoints_with_scores, 0.1)
    
    cv2.imshow('MoveNet Lightning', frame) #name of the cam frame
    
    if cv2.waitKey(10) & 0xFF==ord('q'): #exiting from the opencv : if q pressed quits the cam
        break
        
cap.release()
cv2.destroyAllWindows()

In [9]:
def draw_keypoints(frame, keypoints, confidence_threshold): #confidence threshold will select a minimum confidence for rendering.
    y, x, c = frame.shape 
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1])) #keeping confidence the same(multiplying by 1) so as to compare with the threshold
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 5, (0,255,0), -1) 

In [10]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}
#connections of the coordinates and its colors.
#picked from tensorflow hub 

In [11]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)